In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm



## Part 3: Measuring Geopolitical News

We measure geopolitical risk using the monthly Geopolitical Risk Index (GPR) constructed by Caldara and Iacoviello (2022), which captures geopolitical tensions through newspaper text analysis. The index is publicly available at  
https://www.policyuncertainty.com/gpr.html.

To isolate unexpected updates in geopolitical risk, we construct a shock series from the raw index. Specifically, we model GPR using an AR(1) process and define the geopolitical news shock as the one-step-ahead forecast error:

$$
GPR\_Shock_t = GPR_t - \hat{GPR}_t
$$

where $\hat{GPR}_t$ is the predicted value from the AR(1) model, following the innovation-extraction approach in Engle et al. (2020).


In [5]:

# data cleaning things..
raw = pd.read_excel("data_gpr_export.xls")
df = raw[["month", "GPR"]].copy()
df["month"] = pd.to_datetime(df["month"], dayfirst=True, errors="coerce")
df = df.dropna(subset=["month", "GPR"]).sort_values("month").reset_index(drop=True)

# Make month the index 
df = df.set_index("month")
# definition simple difference shock: GPR_t - GPR_{t-1}
# here I just include a simple difference, to compare the AR to something
df["gpr_change"] = df["GPR"].diff()

# 2) AR(1): GPR_t = c + phi * GPR_{t-1} + u_t
gpr = df["GPR"]
gpr_lag = gpr.shift(1)

df["gpr_ar1_pred"] = np.nan
df["gpr_news_shock"] = np.nan


# Start once we have enough data to estimate, for the AR(1)
# Keep in mind we do not want to use future data to hedge ofc,
# Do we then use an expanding or moving window? Feel free to leave suggestions, gang
min_obs = 24 

for i in range(min_obs, len(df)):
    # Use data up to t-1 to fit AR(1)
    y_train = gpr.iloc[1:i]              
    x_train = gpr_lag.iloc[1:i]          
    train = pd.concat([y_train, x_train], axis=1).dropna()

    y = train.iloc[:, 0]
    x = sm.add_constant(train.iloc[:, 1])

    model = sm.OLS(y, x).fit()

    # One-step-ahead prediction for time t
    x_t = sm.add_constant(pd.Series([gpr_lag.iloc[i]], index=["x"]), has_constant="add")
    # Align names to model params
    x_t = pd.DataFrame({"const": [1.0], train.columns[1]: [gpr_lag.iloc[i]]})

    gpr_hat_t = float(model.predict(x_t)[0])
    shock_t = float(gpr.iloc[i] - gpr_hat_t)

    df.iloc[i, df.columns.get_loc("gpr_ar1_pred")] = gpr_hat_t
    df.iloc[i, df.columns.get_loc("gpr_news_shock")] = shock_t

# Standardize shocks (z-scores)
df["gpr_change_z"] = (df["gpr_change"] - df["gpr_change"].mean()) / df["gpr_change"].std()
df["gpr_news_shock_z"] = (df["gpr_news_shock"] - df["gpr_news_shock"].mean()) / df["gpr_news_shock"].std()

# columns:
# gpr_news_shock   : AR(1) innovation in GPR (raw)
# gpr_news_shock_z : standardized AR(1) GPR shock
# gpr_change      : month-to-month change in GPR
# gpr_change_z    : standardized change in GPR
df


,GPR,gpr_change,gpr_ar1_pred,gpr_news_shock,gpr_change_z,gpr_news_shock_z
month,,,,,,
1985-01-01,102.173378,NaN,NaN,NaN,NaN,NaN
1985-02-01,117.102020,14.928642,NaN,NaN,0.420577,NaN
1985-03-01,124.778152,7.676132,NaN,NaN,0.215553,NaN
1985-04-01,87.929001,-36.849152,NaN,NaN,-1.043155,NaN
1985-05-01,103.262848,15.333847,NaN,NaN,0.432032,NaN
...,...,...,...,...,...,...
2025-08-01,136.759415,1.755157,126.131691,10.627724,0.048170,0.321744
2025-09-01,124.114983,-12.644432,127.436132,-3.321149,-0.358899,-0.084686
2025-10-01,154.425201,30.310219,118.259686,36.165515,0.855407,1.065841


test